In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");
    

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## First question was about in what genres we should invest in USA.

### Options: 

* Hip-Hop
* Punk
* Pop
* Blues

In [3]:
%%sql
SELECT country, billing_country
FROM customer c
INNER JOIN invoice i
ON c.customer_id = i.customer_id
WHERE c.country <> i.billing_country

 * sqlite:///chinook.db
Done.


country,billing_country


### billing_country = country so we can use it to filter USA sales

In [4]:
%%sql
WITH usa_sales AS
    (
    SELECT *
    FROM invoice i
    INNER JOIN invoice_line il
    ON i.invoice_id = il.invoice_id
    WHERE billing_country = 'USA'
    )

SELECT 
    g.name genre_name, 
    SUM(quantity) tracks_sold,
    SUM(quantity) / (SELECT CAST(SUM(quantity) AS Float) FROM usa_sales) total_share
FROM usa_sales u
INNER JOIN track t ON u.track_id = t.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY g.name
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


genre_name,tracks_sold,total_share
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


### Our investigation shows that Punk, Blues and Pop sell the most in the USA, between the options

In [5]:
%%sql
WITH employee_sales AS
    (
    SELECT e.*, SUM(i.total)
    FROM employee e
    INNER JOIN customer c ON e.employee_id = c.support_rep_id
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    GROUP BY e.first_name
    )

SELECT * FROM employee_sales;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email,SUM(i.total)
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com,1731.5100000000039
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com,1584.0000000000034
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com,1393.920000000002


## We can see a positive correlation between time in company and total amount of sales.  

In [6]:
%%sql
WITH
    customer_country AS
    (
    SELECT
        country,
        COUNT(customer_id) customer_count
    FROM customer
    GROUP BY country
    ),
    sales_country AS
    (
    SELECT
        c.country,
        SUM(i.total) sales,
        COUNT(i.invoice_id) orders
    FROM customer c
    INNER JOIN invoice i
    ON c.customer_id = i.customer_id
    GROUP BY country
    ),
    country_filtered AS
    (
    SELECT 
        CASE
            WHEN customer_count = 1 THEN 'Other'
            ELSE cc.country 
        END AS countries,
        CASE
            WHEN customer_count = 1 THEN (SELECT 
                                              SUM(customer_count) 
                                          FROM customer_country
                                          WHERE customer_count = 1
                                         )
            ELSE customer_count
        END AS customers,
        CASE
            WHEN customer_count = 1 THEN (SELECT 
                                              SUM(sc.sales) 
                                          FROM sales_country sc
                                          INNER JOIN customer_country cc
                                          ON sc.country = cc.country
                                          WHERE cc.customer_count = 1
                                         )
            ELSE sales
        END AS total_sales,
        CASE
            WHEN customer_count = 1 THEN (SELECT 
                                              SUM(orders) 
                                          FROM sales_country sc
                                          INNER JOIN customer_country cc
                                          ON sc.country = cc.country
                                          WHERE cc.customer_count = 1
                                         )
            ELSE orders
        END AS total_orders
        FROM customer_country cc
        INNER JOIN sales_country sc
        ON cc.country = sc.country
        GROUP BY countries
    ),
    final_table AS
    (
    SELECT 
        countries,
        customers,
        total_sales,
        total_sales / customers avg_sale_customer,
        total_sales / total_orders avg_order_value
    FROM country_filtered
    ORDER BY 3 DESC
    )
    
SELECT 
    countries,
    customers,
    total_sales,
    avg_sale_customer,
    avg_order_value
FROM 
    (
    SELECT
        ft.*,
        CASE
            WHEN ft.countries = 'Other' THEN 1
            ELSE 0
        END AS sort
    FROM final_table ft
    )
ORDER BY sort;

 * sqlite:///chinook.db
Done.


countries,customers,total_sales,avg_sale_customer,avg_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9399999999998,72.996,7.4485714285714275


In [7]:
%%sql
WITH same_album AS
    (
    SELECT 
        il.invoice_id,
        COUNT(DISTINCT album_id) n_of_albums,
        t.album_id,
        COUNT(il.track_id) n_of_tracks
    FROM invoice_line il
    INNER JOIN track t
    ON il.track_id = t.track_id
    GROUP BY il.invoice_id
    HAVING n_of_albums = 1
    ),
    album_track_count AS
    (
    SELECT
        album_id,
        COUNT(track_id) album_n_tracks
    FROM track
    GROUP BY album_id
    ),
    albums_purchases AS
    (
    SELECT
        sa.invoice_id,
        sa.album_id,
        sa.n_of_tracks,
        atc.album_n_tracks
    FROM same_album sa
    INNER JOIN album_track_count atc
    ON sa.album_id = atc.album_id
    WHERE n_of_tracks = album_n_tracks
    )

SELECT
    COUNT(i.invoice_id) total_orders,
    COUNT(ap.invoice_id) album,
    CAST(COUNT(ap.invoice_id) AS Float) / COUNT(i.invoice_id) percentage_of_albums, 
    COUNT(i.invoice_id) - COUNT(ap.invoice_id) not_album,
    CAST((COUNT(i.invoice_id) - COUNT(ap.invoice_id)) AS Float) / COUNT(i.invoice_id) percentage_of_not_album
    FROM invoice i
    LEFT JOIN albums_purchases ap
    ON i.invoice_id = ap.invoice_id;
    

 * sqlite:///chinook.db
Done.


total_orders,album,percentage_of_albums,not_album,percentage_of_not_album
614,114,0.18566775244299674,500,0.8143322475570033


## Observations:

18,56% of the sales are of full albums. It is not a irelevant part of the total sales so the company should continue to sell full albums

In [12]:
%%sql
SELECT 'Maior Valor' AS Total, MAX(total) FROM invoice
UNION
SELECT 'Meno Valor' AS Total, MIN(total) FROM invoice;

 * sqlite:///chinook.db
Done.


Total,MAX(total)
Maior Valor,23.759999999999998
Meno Valor,0.99
